In [1]:
from abc import ABC, abstractmethod
from datetime import datetime
from datetime import timedelta
from collections import defaultdict
from timeit import default_timer
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import math
import random
from random import sample

In [2]:
class TimeConstraint:

    def __init__(self, end_dt, start_dt=None):
        """
        When end_dt is only given, system will have a max time constraint only.

        When end_dt and start_dt are given, system will have beginning end ending boundary.

        :param end_dt: The maximum limit of the time constraint.
        :param start_dt: The minimum limit of the time constraint.
            Always set start_dt to None if you change the object from time_bin to max_limit.
        """
        self.end_dt = end_dt
        self.start_dt = start_dt

    def is_valid_time_bin(self) -> bool:
        """
        Check whether this TimeConstraint object represents a valid time bin.
        """
        if self.is_time_bin() and (self._end_dt > self._start_dt):
            return True
        return False

    def is_valid_max_limit(self) -> bool:
        """
        Check whether this TimeConstraint represents a valid max time limit.
        """
        if (self._end_dt is not None) and (self._start_dt is None):
            return True

    def is_time_bin(self) -> bool:
        if (self._start_dt is not None) and (self._end_dt is not None):
            return True
        return False

    # Comparing TimeConstraints

    def __eq__(self, other):
        if other is None:
            return False
        return self._start_dt == other.start_dt and self._end_dt == other.end_dt

    def __ne__(self, other):
        if other is None:
            return False
        return self._start_dt != other.start_dt or self._end_dt != other.end_dt

    # Properties

    @property
    def end_dt(self):
        return self._end_dt

    @end_dt.setter
    def end_dt(self, value):
        self._end_dt = value

    @property
    def start_dt(self):
        return self._start_dt

    @start_dt.setter
    def start_dt(self, value):
        self._start_dt = value

    # Printing TimeConstraints

    def __repr__(self):
        return f"(start = {self._start_dt}, end= {self._end_dt})"

    def __str__(self):
        return f"(start = {self._start_dt}, end= {self._end_dt})"


In [3]:
class Cache:

    def __init__(self,
                 is_ratings_cached=False,
                 ratings=None,
                 is_movies_cached=False,
                 movies=None,
                 is_movie_ratings_cached=False,
                 movie_ratings=None,
                 is_user_movie_matrix_cached=False,
                 user_movie_matrix=None,
                 is_user_correlations_cached=False,
                 user_correlations=None,
                 min_common_elements=5,
                 use_avg_ratings_cache=True):
        """ Cached data is only valid when the boolean specifier is True """

        # 30% performance
        self.is_ratings_cached = is_ratings_cached
        self.ratings = ratings

        # 7 fold performance gain on 'movie' related queries
        self.is_movies_cached = is_movies_cached
        self.movies = movies

        self.is_movie_ratings_cached = is_movie_ratings_cached
        self.movie_ratings = movie_ratings

        self.is_user_movie_matrix_cached = is_user_movie_matrix_cached
        self.user_movie_matrix = user_movie_matrix

        self.is_user_correlations_cached = is_user_correlations_cached
        self.user_correlations = user_correlations

        self.min_common_elements = min_common_elements

        # if use avg ratings cache, on average 10 fold performance gain
        self.use_avg_ratings_cache = use_avg_ratings_cache
        if self.use_avg_ratings_cache:
            self.avg_user_ratings = self.create_user_avg_rating_cache()
        else:
            self.avg_user_ratings = None

    def create_user_avg_rating_cache(self):
        if self.is_ratings_cached:
            data = self.ratings
        else:
            data = self.movie_ratings
        return data.groupby('user_id')[['rating']].mean()

    def get_user_corrs(self, min_common_elements, time_constraint=None):
        """
        If cached returns the cache, else none
        :param min_common_elements: min common element in between users in order them to become neighbours
        :param time_constraint: used in temporal caches only, None in this context
        :return: user correlation matrix if cache found, else None
        """
        if self.is_user_correlations_cached:
            if self.min_common_elements == min_common_elements:
                return self.user_correlations
        return None

    # Properties
    @property
    def ratings(self):
        return self._ratings

    @ratings.setter
    def ratings(self, value):
        self._ratings = value

    @property
    def movies(self):
        return self._movies

    @movies.setter
    def movies(self, value):
        self._movies = value

    @property
    def movie_ratings(self):
        return self._movie_ratings

    @movie_ratings.setter
    def movie_ratings(self, value):
        self._movie_ratings = value

    @property
    def user_movie_matrix(self):
        return self._user_movie_matrix

    @user_movie_matrix.setter
    def user_movie_matrix(self, value):
        self._user_movie_matrix = value

    @property
    def user_correlations(self):
        return self._user_correlations

    @user_correlations.setter
    def user_correlations(self, value):
        self._user_correlations = value

    @property
    def min_common_elements(self):
        return self._min_common_elements

    @min_common_elements.setter
    def min_common_elements(self, value):
        self._min_common_elements = value


class TemporalCache(Cache):

    def __init__(self,
                 time_constraint: TimeConstraint,
                 is_ratings_cached=False,
                 ratings=None,
                 is_movies_cached=False,
                 movies=None,
                 is_movie_ratings_cached=False,
                 movie_ratings=None,
                 is_user_movie_matrix_cached=False,
                 user_movie_matrix=None,
                 is_user_correlations_cached=False,
                 user_correlations=None,
                 min_common_elements=5,
                 use_avg_ratings_cache=True,
                 use_bulk_corr_cache=True):

        super().__init__(is_ratings_cached=is_ratings_cached,
                         ratings=ratings,
                         is_movies_cached=is_movies_cached,
                         movies=movies,
                         is_movie_ratings_cached=is_movie_ratings_cached,
                         movie_ratings=movie_ratings,
                         is_user_movie_matrix_cached=is_user_movie_matrix_cached,
                         user_movie_matrix=user_movie_matrix,
                         is_user_correlations_cached=is_user_correlations_cached,
                         user_correlations=user_correlations,
                         min_common_elements=min_common_elements,
                         use_avg_ratings_cache=use_avg_ratings_cache)

        self.time_constraint = time_constraint
        self.use_bulk_corr_cache = use_bulk_corr_cache
        self.user_corrs_in_bulk = None

    def is_temporal_cache_valid(self):
        # No TimeConstraint, valid
        if self._time_constraint is None:
            return True
        # Bin TimeConstraint or Max Limit TimeConstraint, valid
        if self._time_constraint.is_valid_time_bin() or self._time_constraint.is_valid_max_limit():
            return True
        # Else, Not Valid
        return False

    def get_user_corrs_from_bulk(self, min_common_elements, time_constraint, bin_size):
        if ((self.user_corrs_in_bulk is None) or (self.user_corrs_in_bulk is None)
                or (time_constraint is None) or self.min_common_elements != min_common_elements):
            return None

        if time_constraint.is_valid_max_limit():
            return self.user_corrs_in_bulk.get(time_constraint.end_dt.year)

        if bin_size == -1:
            return None

        bins = self.user_corrs_in_bulk.get(bin_size)
        if bins is not None:
            return bins.get(time_constraint.start_dt.year)

    def get_user_corrs(self, min_common_elements, time_constraint=None):
        """
        If cached returns the cache, else none

        :param min_common_elements: min common element in between users in order them to become neighbours
        :param time_constraint: time constraint on user correlations
        :return: user correlation matrix if cache found, else None
        """
        if self.is_user_correlations_cached:
            if self.time_constraint == time_constraint and self.min_common_elements == min_common_elements:
                return self.user_correlations
        return None

    def set_user_corrs(self, user_corrs, min_common_elements, time_constraint):
        # Only set when caching is open for user_correlations
        if self.is_user_correlations_cached:
            self._time_constraint = time_constraint
            self.min_common_elements = min_common_elements
            self.user_correlations = user_corrs

    @property
    def time_constraint(self):
        return self._time_constraint

    @time_constraint.setter
    def time_constraint(self, value):
        self._time_constraint = value


In [95]:
class Accuracy:

    @staticmethod
    def rmse(predictions) -> float:
        if type(predictions) is pd.DataFrame:
            number_of_predictions = 0
            sum_of_square_differences = 0.0
            for row in predictions.itertuples(index=False):
                # row[1] : actual rating, row[0] : prediction
                prediction = row[0]
                if prediction != 0:
                    actual = Accuracy.half_round_rating(row[1])
                    prediction = Accuracy.half_round_rating(prediction)
                    
                    sum_of_square_differences += (actual - prediction) ** 2
                    number_of_predictions += 1
            return sum_of_square_differences / number_of_predictions if number_of_predictions != 0 else 0
        elif type(predictions) is list:
            number_of_predictions = 0
            sum_of_square_differences = 0.0
            for prediction, actual in predictions:
                if prediction != 0:
                    actual = Accuracy.half_round_rating(actual)
                    prediction = Accuracy.half_round_rating(prediction)
                    
                    sum_of_square_differences += (actual - prediction) ** 2
                    number_of_predictions += 1
            return sum_of_square_differences / number_of_predictions if number_of_predictions != 0 else 0
        return 0
    
    @staticmethod
    def threshold_accuracy(predictions) -> float:
        if type(predictions) is pd.DataFrame:
            number_of_predictions = 0
            number_of_hit = 0
            for row in predictions.itertuples(index=False):
                # row[1] : actual rating, row[0] : prediction
                prediction = row[0]
                if prediction != 0:
                    actual = Accuracy.threshold_round_rating(row[1])
                    prediction = Accuracy.threshold_round_rating(prediction)
                    
                    if actual == prediction:
                        number_of_hit += 1
                    number_of_predictions += 1
            return (number_of_hit / number_of_predictions)*100 if number_of_predictions != 0 else 0
        elif type(predictions) is list:            
            number_of_predictions = 0
            number_of_hit = 0
            for prediction, actual in predictions:
                if prediction != 0:
                    actual = Accuracy.threshold_round_rating(actual)
                    prediction = Accuracy.threshold_round_rating(prediction)
                    
                    if actual == prediction:
                        number_of_hit += 1
                        
                    number_of_predictions += 1
            return (number_of_hit / number_of_predictions)*100 if number_of_predictions != 0 else 0
        return 0
    
    @staticmethod
    def threshold_analize(predictions):
        TP, FN, FP, TN = Accuracy.threshold_confusion_matrix(predictions)
        precision = Accuracy.precision(TP, FP)     # also called PPV
        recall = Accuracy.recall(TP, FN)           # also called TPR
        specificity = Accuracy.specificity(FP, TN) # also called TNR
        NPV = Accuracy.negative_predictive_value(FN, TN)
        
        accuracy = Accuracy.accuracy(TP, FN, FP, TN)
        balanced_accuracy = Accuracy.balanced_accuracy(TPR=recall, TNR=specificity)
        informedness = Accuracy.informedness(TPR=recall, TNR=specificity)
        markedness = Accuracy.markedness(PPV=precision, NPV=NPV)
        
        f1 = Accuracy.f_measure(precision, recall)
        mcc = Accuracy.mcc(TP, FN, FP, TN)
        
                
        output = {
                  "accuracy"         :round(accuracy, 3),
                  "balanced_accuracy":round(balanced_accuracy, 3),
                  "informedness"     :round(informedness, 3),
                  "markedness"       :round(markedness, 3),
                  "f1"               :round(f1, 3),
                  "mcc"              :round(mcc, 3),
                  "precision"        :round(precision, 3),
                  "recall"           :round(recall, 3),
                  "specificity"      :round(specificity, 3),
                  "NPV"              :round(NPV, 3)
                 }
        
        return output
    
    @staticmethod
    def analize(predictions):
        """

        https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9
        
        Returns analysis for each class as list
        :return: accuracy, balanced_accuracy, informedness, markedness, f1, mcc, precision, recall, specificity, NPV
        """
        confusion_mtr = Accuracy.confusion_matrix(predictions)
        
        # Use macro averaging (https://stats.stackexchange.com/questions/187768/matthews-correlation-coefficient-with-multi-class)
        precision = [0] * 10 
        recall = [0] * 10 
        specificity = [0] * 10
        NPV = [0] * 10 
        
        accuracy = [0] * 10 
        balanced_accuracy = [0] * 10 
        informedness = [0] * 10 
        markedness = [0] * 10 
        
        f1 = [0] * 10 
        mcc = [0] * 10
        
        for i in range(0, 10):
            TP, FN, FP, TN = Accuracy.confusion_matrix_one_against_all(confusion_mtr, i)
            precision[i] = Accuracy.precision(TP, FP)     # also called PPV
            recall[i] = Accuracy.recall(TP, FN)           # also called TPR
            specificity[i] = Accuracy.specificity(FP, TN) # also called TNR
            NPV[i] = Accuracy.negative_predictive_value(FN, TN)

            accuracy[i] = Accuracy.accuracy(TP, FN, FP, TN)
            balanced_accuracy[i] = Accuracy.balanced_accuracy(TPR=recall[i], TNR=specificity[i])
            informedness[i] = Accuracy.informedness(TPR=recall[i], TNR=specificity[i])
            markedness[i] = Accuracy.markedness(PPV=precision[i], NPV=NPV[i])

            f1[i] = Accuracy.f_measure(precision[i], recall[i])
            mcc[i] = Accuracy.mcc(TP, FN, FP, TN)
        
        output = {
                  "accuracy"         :Accuracy.round_list_elements(accuracy, 3),
                  "balanced_accuracy":Accuracy.round_list_elements(balanced_accuracy, 3),
                  "informedness"     :Accuracy.round_list_elements(informedness, 3),
                  "markedness"       :Accuracy.round_list_elements(markedness, 3),
                  "f1"               :Accuracy.round_list_elements(f1, 3),
                  "mcc"              :Accuracy.round_list_elements(mcc, 3),
                  "precision"        :Accuracy.round_list_elements(precision, 3),
                  "recall"           :Accuracy.round_list_elements(recall, 3),
                  "specificity"      :Accuracy.round_list_elements(specificity, 3),
                  "NPV"              :Accuracy.round_list_elements(NPV, 3)
                 }
        
        return output
    
    @staticmethod
    def round_list_elements(l, precision):
        """
        :param l: list of floats
        :param precision: precision after dot
        """
        return [ round(x, precision) for x in l ]
    
    @staticmethod
    def accuracy_multi_class(confusion_mtr):
        length = len(confusion_mtr)
        numenator = 0
        denuminator = 0 
        for i in range(length):
            for j in range(length):
                temp = confusion_mtr[i][j]
                denuminator += temp
                if i == j:
                    numenator += temp
        return numenator / denominator
    
    @staticmethod
    def accuracy(TP, FN, FP, TN):
        return  (TP + TN) / (TP + FN + FP + TN)
    
    @staticmethod
    def balanced_accuracy(TPR, TNR):
        """
        :param TPR : True Positive Rate or recall or sensitivity
        :param TNR : True Negative Rate or specificity or  selectivity
        """
        return (TPR + TNR) / 2
    
    @staticmethod
    def informedness(TPR, TNR):
        """
        :param TPR : True Positive Rate or recall or sensitivity
        :param TNR : True Negative Rate or specificity or  selectivity
        """
        return TPR + TNR - 1
    
    @staticmethod
    def markedness(PPV, NPV):
        """
        :param PPV: Positive Predictive Value also known as precision
        :param NPV: Negative Predictive Value
        """
        return PPV + NPV - 1 
    
    @staticmethod
    def precision(TP, FP):
        """
        Also called as precision or positive predictive value (PPV)
        
        Precision = TP / (TP + FP) for binary class
        Precision = TP / (All Predicted Positive) for multi class
        """
        denuminator = TP + FP
        return TP / denuminator if denuminator != 0 else 0

    @staticmethod
    def negative_predictive_value(FN, TN):
        """     
        NPV = TN / (TN + FN) for binary class
        NPV = TN / (All Predicted Negative) for multi class
        """
        denuminator = TN + FN
        return TN / denuminator if denuminator != 0 else 0    
    
    @staticmethod
    def recall(TP, FN):
        """
        Also called as sensitivity, recall, hitrate, or true positive rate(TPR)
        Recall = TP / (TP + FN) for binary class
        Recall = TP / (All Actual Positive) for multi class
        """
        denuminator = TP + FN
        return TP / denuminator if denuminator != 0 else 0    
    
    @staticmethod
    def specificity(FP, TN):
        """
        Also called as specificity, selectivity or true negative rate (TNR)
        specificity = TN / (TP + FN) for binary class
        specificity = TN / (All Actual Negative) for multi class
        """
        denuminator = FP + TN 
        return TN / denuminator if denuminator != 0 else 0 
    
    @staticmethod
    def f_measure(precision, recall):
        """
        F-Measure is the harmonic mean of the precision and recall.
        """
        sum_of_both = precision + recall
        return (2 * precision * recall) / sum_of_both if sum_of_both != 0 else 0
    
    @staticmethod
    def mcc(TP, FN, FP, TN):
        """
        MCC(Matthews Correlation Coefficient)
        """
        # Calulate Matthews Correlation Coefficient
        numenator   = (TP * TN) - (FP * FN) 
        denominator = (TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)
        denominator = math.sqrt(denominator) if denominator > 0 else 0
        return numenator / denominator if denominator != 0 else 0
  
# deprecated -> now, we use macro averaging instad of multi class versions of metrics
#     @staticmethod
#     def mcc_multi_class(confusion_mtr):
#         # https://en.wikipedia.org/wiki/Matthews_correlation_coefficient#Multiclass_case
#         # https://stats.stackexchange.com/questions/187768/matthews-correlation-coefficient-with-multi-class
#         length = len(confusion_mtr)
#         numenator = 0
#         for k in range(length):
#             for l in range(length):
#                 for m in range(length):
#                     numenator += confusion_mtr[k][k] * confusion_mtr[l][m]
#                     numenator -= confusion_mtr[k][l] * confusion_mtr[m][k]

#         denuminator_1 = 0
#         for k in range(length):
#             denuminator_part_1 = 0
#             for l in range(length):
#                 denuminator_part_1 += confusion_mtr[k][l]

#             denuminator_part_2 = 0
#             for f in range(length):
#                 if f != k:
#                     for g in range(length):
#                         denuminator_part_2 += confusion_mtr[f][g]
#             denuminator_1 += denuminator_part_1 * denuminator_part_2

#         denuminator_2 = 0
#         for k in range(length):
#             denuminator2_part_1 = 0
#             for l in range(length):
#                 denuminator2_part_1 += confusion_mtr[l][k]

#             denuminator2_part_2 = 0
#             for f in range(length):
#                 if f != k:
#                     for g in range(length):
#                         denuminator2_part_2 += confusion_mtr[g][f]
#             denuminator_2 += denuminator2_part_1 * denuminator2_part_2
#         return numenator / math.sqrt(denuminator_1) * math.sqrt(denuminator_2)
    
    @staticmethod
    def confusion_matrix_one_against_all(confusion_mtr, class_i):
        """
        Create binary confusion matrix out of multi-class confusion matrix
        
        Positive Class: class_i
        Negative Class: non class_i
                
        TP: True Positive   FN: False Negative
        FP: False Positive  TN: True Negative
        
        "TP of Class_1" is all Class_1 instances that are classified as Class_1.
        "TN of Class_1" is all non-Class_1 instances that are not classified as Class_1.
        "FP of Class_1" is all non-Class_1 instances that are classified as Class_1.
        "FN of Class_1" is all Class_1 instances that are not classified as Class_1.
        # https://www.researchgate.net/post/How_do_you_measure_specificity_and_sensitivity_in_a_multiple_class_classification_problem
        
        --> Input matrix
                 | 0 Prediction | 1 Prediction | 2 Prediction | .....
        0 Class  |     T0       |     ..       |      ..      |
        1 Class  |     ..       |     T1       |      ..      | 
        2 Class  |     ..       |     ..       |      T2      |
        
        --> Output matrix
        
                        | Positive Prediction | Negative Prediction
        Positive Class  |       TP            |       FN
        Negative Class  |       FP            |       TN
        
        :param confusion_mtr: 10 class confusion matrix designed for movielens
        :param class_i: index of the class we are interested in(0-9)
        :return: TP, FN, FP, TN
        """
        length = len(confusion_mtr)

        TP = confusion_mtr[class_i][class_i] 
        
        actual_class_i_count = 0 
        for i in range(length):  # sum of the row
            actual_class_i_count += confusion_mtr[class_i][i]
        FN = actual_class_i_count - TP
        
        predicted_class_i_count = 0
        for i in range(length): # sum of the column
            predicted_class_i_count += confusion_mtr[i][class_i]
        FP = predicted_class_i_count - TP
        
        # sum of matrix
        sum_of_matrix = np.sum(confusion_mtr)
        # TN is found by summing up all values except the row and column of the class 
        TN = sum_of_matrix - predicted_class_i_count - actual_class_i_count - TP 
        
        return TP, FN, FP, TN
        
    @staticmethod
    def confusion_matrix(predictions):
        """
        Create confusion matrix and then return TP, FN, FP, TN

        0 Class: 0.5
        1 Class: 1
        2 Class: 1.5
        3 Class: 2
        4 Class: 2.5
        5 Class: 3
        6 Class: 3.5
        7 Class: 4
        8 Class: 4.5
        9 Class: 5

        T0: True 0
        F0: False 0
        T1: True 1
        F1: False 1
        ...

                 | 0 Prediction | 1 Prediction | 2 Prediction | .....
        0 Class  |     T0       |     ..       |      ..      |
        1 Class  |     ..       |     T1       |      ..      | 
        2 Class  |     ..       |     ..       |      T2      |
        ...
        """
        # Create multiclass confusion matrix

        conf_mtr = np.zeros( (10,10) )

        for prediction in predictions:
            predicted = Accuracy.half_round_rating(prediction[0])
            actual    = Accuracy.half_round_rating(prediction[1])

            predicted_class_index = int( (predicted * 2) - 1 )
            actual_class_index = int( (actual * 2) - 1 )

            conf_mtr[actual_class_index][predicted_class_index] += 1

        return conf_mtr
    
    @staticmethod
    def threshold_confusion_matrix(predictions):
        """
        Create confusion matrix and then return TP, FN, FP, TN

        Positive Class: 5
        Negative Class: 0.5

        TP: True Positive
        TN: True Negative
        FP: False Positive
        FN: False Negative

                        | Positive Prediction | Negative Prediction
        Positive Class  |       TP            |       FN
        Negative Class  |       FP            |       TN

        """
        # Create confusion matrix
        TP = 0
        TN = 0
        FP = 0
        FN = 0
        for prediction in predictions:
            predicted = Accuracy.threshold_round_rating(prediction[0])
            actual = Accuracy.threshold_round_rating(prediction[1])
            if predicted == 5 and actual == 5:
                TP += 1
            elif predicted == 5 and actual == 0.5:
                FP += 1
            elif predicted == 0.5 and actual == 0.5:
                TN += 1
            elif predicted == 0.5 and actual == 5:
                FN += 1
        return TP, FN, FP, TN

    @staticmethod
    def half_round_rating(rating):
        """
        Round ratings to the closest match in the movielens dataset
        For ex.
          ratings between 2 and 2.25 -> round to 2
          ratings between 2.25 and 2.5 -> round to 2.5
          ratings between 2.5 and 2.75 -> round to 2.5
          ratings between 2.75 and 3 -> round to 3

        """
        floor_value = math.floor(rating)
        if(rating > floor_value + 0.75):
            return floor_value + 1
        elif(rating > floor_value + 0.5 or rating > floor_value + 0.25):
            return floor_value + 0.5
        else:
            return floor_value
    
    @staticmethod
    def threshold_round_rating(rating):
        """
        Round ratings to the closest match in threshold fashion
          ratings between 0.5 and 2.25 -> round to 0.5
          ratings between 2.25 and 5 -> round to 5
        """
        if (0.5 <= rating < 3.5):
            return 0.5
        elif (3.5 <= rating <= 5):
            return 5
        else:
            return 0

In [5]:
class Dataset(ABC):
    @staticmethod
    @abstractmethod
    def load():
        """ Every subclass must provide static load method"""
        pass


class MovieLensDataset(Dataset):
    def __init__(self,
                 ratings_col_names=('user_id', 'item_id', 'rating', 'timestamp'),
                 ratings_path=r'C:\Users\Yukawa\datasets\ml-latest-small\ratings.csv',
                 movies_col_names=('item_id', 'title', 'genres'),
                 movies_path=r'C:\Users\Yukawa\datasets\ml-latest-small\movies.csv',
                 is_ratings_cached=True,
                 is_movies_cached=True):
        Dataset.__init__(self)
        self.is_ratings_cached = is_ratings_cached
        self.is_movies_cached = is_movies_cached
        self.ratings = MovieLensDataset.load_ratings(ratings_path,
                                                     ratings_col_names) if self.is_ratings_cached else None
        self.movies = MovieLensDataset.load_movies(movies_path,
                                                   movies_col_names) if self.is_movies_cached else None

    @staticmethod
    def load_movies(movies_path,
                    movies_col_names=('item_id', 'title', 'genres')):
        if not os.path.isfile(movies_path) or not movies_col_names:
            return None

        # read movies
        movies = pd.read_csv(movies_path, sep=',', header=1, names=movies_col_names)

        # Extract Movie Year
        movies['year'] = movies.title.str.extract("\((\d{4})\)", expand=True)
        movies.year = pd.to_datetime(movies.year, format='%Y')
        movies.year = movies.year.dt.year  # As there are some NaN years, resulting type will be float (decimals)

        # Remove year part from the title
        movies.title = movies.title.str[:-7]

        return movies

    @staticmethod
    def load_ratings(ratings_path,
                     ratings_col_names=('user_id', 'item_id', 'rating', 'timestamp')):
        if not os.path.isfile(ratings_path) or not ratings_col_names:
            return None

        # read ratings
        ratings = pd.read_csv(ratings_path, sep=',', header=1, names=ratings_col_names)

        # Convert timestamp into readable format
        ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s', origin='unix')

        return ratings

    @staticmethod
    def create_movie_ratings(ratings, movies):
        return pd.merge(ratings, movies, on='item_id')

    @staticmethod
    def load(ratings_col_names=('user_id', 'item_id', 'rating', 'timestamp'),
             ratings_path=r'C:\Users\Yukawa\datasets\ml-latest-small\ratings.csv',
             movies_col_names=('item_id', 'title', 'genres'),
             movies_path=r'C:\Users\Yukawa\datasets\ml-latest-small\movies.csv'
             ):
        # Load movies
        movies = MovieLensDataset.load_movies(movies_path=movies_path, movies_col_names=movies_col_names)
        # Load ratings
        ratings = MovieLensDataset.load_ratings(ratings_path=ratings_path, ratings_col_names=ratings_col_names)

        # Merge the ratings and movies
        movie_ratings = pd.merge(ratings, movies, on='item_id')

        return movie_ratings

In [6]:
class TemporalPearson:

    def __init__(self, cache: TemporalCache, time_constraint: TimeConstraint = None, min_common_elements: int = 5):
        self.time_constraint = time_constraint
        self.cache = cache
        self.min_common_elements = min_common_elements
        #from .trainset import TrainsetUser, TrainsetMovie
        self.trainset_user = TrainsetUser(cache=self.cache)
        self.trainset_movie = TrainsetMovie(cache=self.cache)

    def mean_centered_pearson(self, user_id, movie_id, k_neighbours: pd.DataFrame) -> float:
        """
        Calculate Mean Centered Prediction

        :param user_id: user of interest
        :param movie_id: the movie's rating is the one we we want to predict
        :param k_neighbours: k nearest neighbours in DataFrame where index user_id, column correlation in between.
        :return: Prediction rating
        """
        # If a movie with movie_id not exists, predict 0
        if self.trainset_movie.get_movie(movie_id=movie_id).empty:
            return 0

        if k_neighbours is None or k_neighbours.empty:
            return 0

        user_avg_rating = self.trainset_user.get_user_avg(user_id=user_id)

        weighted_sum = 0.0
        sum_of_weights = 0.0
        for neighbour_id, data in k_neighbours.iterrows():
            # Get each neighbour's correlation 'user_id' and her rating to 'movie_id'
            neighbour_corr = data['correlation']
            neighbour_rating = self.trainset_movie.get_movie_rating(movie_id=movie_id, user_id=neighbour_id)
            # If the neighbour doesnt give rating to the movie_id, pass this around of the loop
            if neighbour_rating == 0:
                continue
            neighbour_avg_rating = self.trainset_user.get_user_avg(user_id=neighbour_id)
            neighbour_mean_centered_rating = neighbour_rating - neighbour_avg_rating
            # Calculate Weighted sum and sum of weights
            weighted_sum += neighbour_mean_centered_rating * neighbour_corr
            sum_of_weights += neighbour_corr

        # Predict
        if sum_of_weights != 0:
            prediction_rating = user_avg_rating + (weighted_sum / sum_of_weights)
        else:
            prediction_rating = 0  # In this case, none of the neighbours have given rating to 'the movie'

        return prediction_rating

    def get_corr_matrix(self, bin_size=-1):
        user_corrs = None
        # if valid cache found, try to get user corrs from there
        if self.cache.is_temporal_cache_valid():
            # First check user-correlations
            user_corrs = self.cache.get_user_corrs(self.min_common_elements, self.time_constraint)
            if user_corrs is not None:
                return user_corrs
            # Then check bulk-user-correlations
            user_corrs = self.cache.get_user_corrs_from_bulk(time_constraint=self.time_constraint,
                                                             min_common_elements=self.min_common_elements,
                                                             bin_size=bin_size)
            if user_corrs is not None:
                return user_corrs

        # here, if cache not found or no cache match

        # Create user correlations
        user_corrs = TemporalPearson.create_user_corrs(movie_ratings=self.cache.movie_ratings,
                                                       time_constraint=self.time_constraint,
                                                       min_common_elements=self.min_common_elements)
        # Cache the user_corrs
        self.cache.set_user_corrs(user_corrs=user_corrs,
                                  min_common_elements=self.min_common_elements,
                                  time_constraint=self.time_constraint)

        return user_corrs

    @staticmethod
    def create_user_corrs(movie_ratings, time_constraint: TimeConstraint, min_common_elements):
        # by default movie_ratings is for no time constraint
        # with these controls change the time constraint of the movie_ratings
        if time_constraint is not None:
            if time_constraint.is_valid_max_limit():
                movie_ratings = movie_ratings[movie_ratings.timestamp < time_constraint.end_dt]
            elif time_constraint.is_valid_time_bin():
                movie_ratings = movie_ratings[(movie_ratings.timestamp >= time_constraint.start_dt)
                                              & (movie_ratings.timestamp < time_constraint.end_dt)]

        user_movie_matrix = movie_ratings.pivot_table(index='title', columns='user_id', values='rating')
        return user_movie_matrix.corr(method="pearson", min_periods=min_common_elements)

    def cache_user_corrs_in_bulk_for_max_limit(self, time_constraint: TimeConstraint, min_year, max_year):
        """
        Cache user correlations by changing year of the time_constraint
        for each year in between min_year and max_year(not included)

        :param time_constraint: time_constraint apply
        :param min_year: start of the range
        :param max_year: end of the range
        """

        if self.cache.use_bulk_corr_cache:
            if time_constraint is not None and time_constraint.is_valid_max_limit():
                self.cache.user_corrs_in_bulk = dict()
                for year in range(min_year, max_year):
                    time_constraint.end_dt = time_constraint.end_dt.replace(year=year)
                    corrs = TemporalPearson.create_user_corrs(self.cache.movie_ratings, time_constraint,
                                                              self.min_common_elements)
                    self.cache.user_corrs_in_bulk[year] = corrs
            else:
                raise Exception("Trying to cache user correlations in bulk for max_limit "
                                "but start time is not max_limit!")
        else:
            raise Exception("Trying to create bulk corr cache when use_bulk_corr_cache is False")

    def cache_user_corrs_in_bulk_for_time_bins(self, time_constraint: TimeConstraint, min_year, max_year,
                                               min_time_bin_size=2, max_time_bin_size=10):
        if self.cache.use_bulk_corr_cache:
            if time_constraint is not None and time_constraint.is_valid_time_bin():
                del self.cache.user_corrs_in_bulk    # invalidate old cache
                self.cache.user_corrs_in_bulk = dict()
                for time_bin_size in range(min_time_bin_size, max_time_bin_size):
                    self.cache.user_corrs_in_bulk[time_bin_size] = dict()
                    for shift in range(0, time_bin_size):
                        curr_year = min_year + shift
                        while (curr_year + time_bin_size) < max_year:
                            time_constraint = TimeConstraint(start_dt=datetime(curr_year, 1, 1),
                                                             end_dt=datetime(curr_year + time_bin_size, 1, 1))
                            corrs = TemporalPearson.create_user_corrs(self.cache.movie_ratings,
                                                                      time_constraint,
                                                                      self.min_common_elements)
                            self.cache.user_corrs_in_bulk[time_bin_size][curr_year] = corrs
                            curr_year += time_bin_size
        else:
            raise Exception("Trying to create bulk corr cache when use_bulk_corr_cache is False")

    @property
    def time_constraint(self):
        return self._time_constraint

    @time_constraint.setter
    def time_constraint(self, value):
        self._time_constraint = value


In [7]:
class TrainsetUser:

    def __init__(self, cache: Cache):
        """
        :param cache: Input cache must have movie_ratings not None !
        """
        self.cache = cache

        if not self.cache.is_movie_ratings_cached:
            raise Exception("'movie_ratings' has not been cached !")

    def get_users(self):
        """
        Get list of unique 'user_id's

        Since MovieLens Have 'user_id's from 0 to 610 without any missing user, for now sending that directly
        Uncomment the other lines later

        :return: the ids of the users found in movie_ratings
        """
        #
        # if self.cache.is_ratings_cached:
        #     data = self.cache.ratings
        # else:
        #     data = self.cache.movie_ratings
        #
        # return pd.unique(data['user_id'])
        return range(0, 611)

    def get_active_users(self, n=10) -> pd.DataFrame:
        """
        Get Users in sorted order where the first one is the one who has given most ratings.

        :param n: Number of users to retrieve.
        :return: user DataFrame with index of 'user_id' and columns of ['mean_rating', 'No_of_ratings'] .
        """

        if self.cache.is_ratings_cached:                         # 30% faster than other choice
            data = self.cache.ratings
        else:
            data = self.cache.movie_ratings

        active_users = pd.DataFrame(data.groupby('user_id')['rating'].mean())
        active_users['No_of_ratings'] = pd.DataFrame(data.groupby('user_id')['rating'].count())
        active_users.sort_values(by=['No_of_ratings'], ascending=False, inplace=True)
        active_users.columns = ['mean_rating', 'No_of_ratings']
        return active_users.head(n)

    def get_random_users(self, n=1):
        """
        Get list of random n number of 'user_id's

        :param n: Number of random users
        :return: List of random 'user_id's
        """

        return random.choices(population=self.get_users(), k=n)

    def get_user_ratings(self, user_id: int) -> pd.DataFrame:
        """
        Get all the ratings given by of the chosen users

        :param user_id: id of the chosen user
        :return: Ratings given by the 'user_id'
        """
        if self.cache.is_ratings_cached:                         # 2.2x faster than other choice
            data = self.cache.ratings
        else:
            data = self.cache.movie_ratings

        return data.loc[data['user_id'] == user_id]

    def get_user_avg(self, user_id: int):

        if self.cache.use_avg_ratings_cache:
            avg_user_rating = self.cache.avg_user_ratings.loc[user_id]
            return avg_user_rating[0] if not avg_user_rating.empty else 0

        user_ratings = self.get_user_ratings(user_id=user_id)
        return user_ratings.rating.mean() if not user_ratings.empty else 0

    def get_timestamp(self, user_id: int, movie_id: int):
        """
        Get the timestamp of the given rating

        :param user_id: the users whose rating timestamp we are searching
        :param movie_id: id of the movie that the user gave the rating
        :return: if found the datetime object otherwise None
        """

        if self.cache.is_ratings_cached:
            data = self.cache.ratings
        else:
            data = self.cache.movie_ratings

        timestamp = data.loc[(data['user_id'] == user_id) & (data['item_id'] == movie_id)]
        return timestamp.values[0, 3] if not timestamp.empty else None

    def get_first_timestamp(self):
        if self.cache.is_ratings_cached:
            data = self.cache.ratings
        else:
            data = self.cache.movie_ratings
        return data['timestamp'].min()

    def get_user_avg_timestamp(self, user_id: int):
        user_ratings = self.get_user_ratings(user_id=user_id)
        return user_ratings.timestamp.mean() if not user_ratings.empty else 0

    # TODO: Later, create TemporalDatasetUser, and put this method into that one
    def get_user_ratings_at(self, user_id: int, at: datetime) -> pd.DataFrame:
        """
        Get user ratings up until the given datetime
        :param user_id: id of the chosen user
        :param at: only those ratings that are before this date will be taken into account
        :return: Ratings given by the 'user_id' before given datetime
        """

        if self.cache.is_ratings_cached:
            data = self.cache.ratings
        else:
            data = self.cache.movie_ratings

        return data.loc[(data['user_id'] == user_id) & (data.timestamp < at)]

    # TODO: Later, create TemporalDatasetUser, and put this method into that one
    def get_user_avg_at(self, user_id: int, at: datetime):
        user_ratings = self.get_user_ratings_at(user_id, at)
        return user_ratings.rating.mean() if not user_ratings.empty else 0


class TrainsetMovie:

    def __init__(self, cache: Cache):
        """
        :param cache: Input cache must have movie_ratings not None !
        """
        self.cache = cache

        if not self.cache.is_movie_ratings_cached:
            raise Exception("'movie_ratings' has not been cached !")

    def get_movie(self, movie_id) -> pd.DataFrame:
        """
        Get Movie Record

        :return: DataFrame which contains the given 'movie_id's details. If not found empty DataFrame .
        """
        if self.cache.is_movies_cached:
            return self.cache.movies.loc[self.cache.movies['item_id'] == movie_id]
        return self.cache.movie_ratings.loc[self.cache.movie_ratings['item_id'] == movie_id]

    def get_movies(self):
        """
        Get list of unique 'item_id's or in other words the movies.

        :return: List of movie ids
        """

        if self.cache.is_movies_cached:
            return self.cache.movies['item_id'].values.tolist()

        return pd.unique(self.cache.movie_ratings['item_id'])

    def get_random_movies(self, n=10):
        """
        Get list of random n number of 'item_id's or in other words the movies

        :param n: Number of random movies
        :return: List of random 'movie_id's
        """
        return random.choices(population=self.get_movies(), k=n)

    def get_movies_watched(self, user_id: int, time_constraint: TimeConstraint = None) -> pd.DataFrame:
        """
        Get all the movies watched by the chosen user.

        :param user_id: the user that we want to get the movies he-she has watched.
        :param time_constraint: type of the time constraint.
        :return: DataFrame of all movies watched with 'item_id', 'rating' columns
        """

        movie_ratings = self.cache.movie_ratings

        if time_constraint is None:
            return movie_ratings.loc[(movie_ratings['user_id'] == user_id)][['item_id', 'rating']]

        if time_constraint.is_valid_max_limit():
            return movie_ratings.loc[(movie_ratings['user_id'] == user_id)
                                     & (movie_ratings.timestamp < time_constraint.end_dt)][['item_id', 'rating']]
        elif time_constraint.is_valid_time_bin():
            return movie_ratings.loc[(movie_ratings['user_id'] == user_id)
                                     & (movie_ratings.timestamp >= time_constraint.start_dt)
                                     & (movie_ratings.timestamp < time_constraint.end_dt)][['item_id', 'rating']]
        raise Exception("Undefined time_constraint is given!")

    def get_movie_rating(self, movie_id: int, user_id: int) -> int:
        """
        Get the movie rating taken by the chosen user

        :param movie_id: the movie chosen movie's id
        :param user_id: id of the chosen user
        :return: Rating given by user. If not found, returns 0
        """

        if self.cache.is_ratings_cached:
            data = self.cache.ratings
        else:
            data = self.cache.movie_ratings

        movie_rating = data.loc[(data['user_id'] == user_id) & (data['item_id'] == movie_id)]
        return movie_rating.values[0, 2] if not movie_rating.empty else 0

    def get_random_movie_watched(self, user_id: int) -> int:
        """
        Get random movie id watched.

        :param user_id: User of interest
        :return:  movie_id or item_id of the random movie watched by the user.
                  In case non-valid user_id supplied then returns 0
        """
        movies_watched = self.get_movies_watched(user_id=user_id)
        return random.choice(movies_watched['item_id'].values.tolist()) if not movies_watched.empty else 0

    def get_random_movies_watched(self, user_id: int, n=2) -> pd.DataFrame:
        """
        Get random n movies watched by the user. Only use when n > 2

        Use get_random_movie_watched if n=1 since that one 2 fold faster.

        :param user_id: the user of interest
        :param n: number of random movies to get
        :return: DataFrame of movies, if none found then empty DataFrame
        """
        movies_watched = self.get_movies_watched(user_id=user_id)
        return random.choices(population=movies_watched['item_id'].values.tolist(),
                              k=n) if not movies_watched.empty else movies_watched

    def get_random_movie_per_user(self, user_id_list):
        """
        Get random movie for each user given in the 'user_id_list'

        :param user_id_list: List of valid user_ids
        :return: List of (user_id, movie_id) tuples
                where each movie_id is randomly chosen from watched movies of the user_id .
                In case any one of the user_id's supplies invalid, then the movie_id will be 0 for that user.
        """
        user_movie_list = list()
        for user_id in user_id_list:
            user_movie_list.append((user_id, self.get_random_movie_watched(user_id=user_id)))
        return user_movie_list


class Trainset:
    def __init__(self, cache: TemporalCache, min_common_elements: int = 5):
        self.cache = cache
        self.min_common_elements = min_common_elements
        self.similarity = TemporalPearson(time_constraint=None, cache=self.cache)

        if not self.cache.is_movie_ratings_cached:
            raise Exception("'movie_ratings' has not been cached !")

        self.trainset_movie = TrainsetMovie(cache=cache)
        self.trainset_user = TrainsetUser(cache=cache)

        # if caching is allowed, create user correlations cache
        self.similarity.get_corr_matrix()

    def predict_movies_watched(self, user_id, n=10, k=10, time_constraint=None) -> pd.DataFrame:
        """

        :param user_id: user of interest
        :param n: Number of movies to predict
        :param k: k neighbours to take into account
        :param time_constraint: When calculating k neighbours,
                                only those that comply to time_constraints will be taken into account.
        :return: DataFrame of Predictions where columns = ['prediction', 'rating'] index = 'movie_id'
        """
        # Get all movies watched by a user
        movies_watched = self.trainset_movie.get_movies_watched(user_id=user_id)

        if movies_watched.empty:
            return None

        predictions = list()
        number_of_predictions = 0
        for row in movies_watched.itertuples(index=False):
            prediction = self.predict_movie(user_id=user_id, movie_id=row[0],
                                            time_constraint=time_constraint, k=k)
            if number_of_predictions == n:
                break
            predictions.append([prediction, row[1], row[0]])
            number_of_predictions += 1

        predictions_df = pd.DataFrame(predictions, columns=['prediction', 'rating', 'movie_id'])
        predictions_df.movie_id = predictions_df.movie_id.astype(int)
        return predictions_df.set_index('movie_id')

    def predict_movie(self, user_id, movie_id, k=10, time_constraint=None, bin_size=-1):
        prediction = self.similarity.mean_centered_pearson(user_id=user_id,
                                                           movie_id=movie_id,
                                                           k_neighbours=
                                                           self.get_k_neighbours(user_id, k=k,
                                                                                 time_constraint=time_constraint,
                                                                                 bin_size=bin_size)
                                                           )        
        return prediction if prediction <= 5 else 5

    def get_k_neighbours(self, user_id, k=10, time_constraint: TimeConstraint = None, bin_size=-1):
        """
        :param user_id: the user of interest
        :param k: number of neighbours to retrieve
        :param time_constraint: time constraint when choosing neighbours
        :param bin_size: Used when using time_bins, in order to select bin from cache
        :return: Returns the k neighbours and correlations in between them. If no neighbours found, returns None
                 DataFrame which has 'Correlation' column and 'user_id' index.
        """
        self.similarity.time_constraint = time_constraint
        user_corr_matrix = self.similarity.get_corr_matrix(bin_size=bin_size)

        # Exit if matrix is None, no user found in self.cache.movie_ratings, something is wrong
        if user_corr_matrix is None:
            return None

        # Get the chosen 'user_id's correlations
        user_correlations = user_corr_matrix.get(user_id)
        if user_correlations is None:
            return None

        # Drop any null, if found
        user_correlations.dropna(inplace=True)
        # Create A DataFrame from not-null correlations of the 'user_id'
        users_alike = pd.DataFrame(user_correlations)
        # Rename the only column to 'correlation'
        users_alike.columns = ['correlation']

        # Sort the user correlations in descending order
        #     so that first one is the most similar, last one least similar
        users_alike.sort_values(by='correlation', ascending=False, inplace=True)

        # Eliminate Correlation to itself by deleting first row,
        #     since biggest corr is with itself it is in first row
        return users_alike.iloc[1:k+1]


In [8]:
class Evaluator:

    def __init__(self, trainset: Trainset):
        self.trainset = trainset

    def evaluate_best_max_year_in_bulk(self, n,
                                       n_users, n_movies, k=10,
                                       min_year=-1,
                                       max_year=-1) -> dict:
        """
        Evaluate and collect data about best max year constraint which can be put instead of no constraint.

        This method calls 'evaluate_best_max_year_constraint' method 'n' times.
        Takes required precautions for bulk calling.

        :param n: Number of runs that we run the evaluate_best_max_year_constraint() method
        :param n_users: Number of users to check
        :param n_movies: Number of movies per user to check
        :param k: Number of neighbours of each user to take into account when making prediction
        :param min_year: First year to evaluate
        :param max_year: Last year to evaluate
        :return: (no_constrain_rmse_data, best_year_constraint_results)
        """
        if min_year == -1:
            min_year = self.trainset.trainset_user.get_first_timestamp().year

        if max_year == -1:
            max_year = datetime.now().year

        time_constraint = TimeConstraint(end_dt=datetime(year=min_year, month=1, day=1))
        # Create cache if bulk_corr_cache is allowed
        self.trainset.similarity.cache_user_corrs_in_bulk_for_max_limit(time_constraint,
                                                                        min_year=min_year,
                                                                        max_year=max_year)
        
        run_results = dict()
        for i in range(n):
            run_results[i] = self.evaluate_best_max_year_constraint(n_users=n_users, n_movies=n_movies, k=k,
                                                                    min_year=min_year, max_year=max_year,
                                                                    create_cache=False,)

        return run_results

    def evaluate_best_max_year_constraint(self, n_users, n_movies, k,
                                          max_diff=0.1,
                                          min_year=-1, max_year=-1,
                                          create_cache=True) -> defaultdict:
        """
        Evaluate the max_year constraint for evaluate_max_year_constraint method.

        :param max_diff: maximum difference between rmse when no constraint and with given year constraint.
        :param n_users: Number of users to evaluate
        :param n_movies: Number of movies per user to evaluate
        :param k: Number of neighbours of each user to take into account when making prediction
        :param min_year: First year to evaluate
        :param max_year: Last year to evaluate
        :param create_cache: create cache before running. For bulk callers.
        :return: Votes for years where each year got its vote
                 when rmse is less than 'max_diff' in between no constraint and year constraint
        """

        if min_year == -1:
            min_year = self.trainset.trainset_user.get_first_timestamp().year

        if max_year == -1:
            max_year = datetime.now().year

        if n_users > 600:
            user_list = self.trainset.trainset_user.get_users()  # No need to random selection, get all users
        else:
            user_list = self.trainset.trainset_user.get_random_users(n=n_users)  # Select random n users

        # Calculate RMSE With No Constraint
        no_constraint_data = dict()
        for user_id in user_list:
            rmse = Accuracy.rmse(self.trainset.predict_movies_watched(user_id, n_movies, k))
            no_constraint_data[user_id] = rmse

        # # Calculate RMSE With Time Constraint

        # Cache all years before processing
        time_constraint = TimeConstraint(end_dt=datetime(year=min_year, month=1, day=1))
        # Create cache if bulk_corr_cache is allowed
        if create_cache:
            self.trainset.similarity.cache_user_corrs_in_bulk_for_max_limit(time_constraint,
                                                                            min_year=min_year,
                                                                            max_year=max_year)
        # Votes to years is stored inside time_constraint_data
        time_constraint_data = defaultdict(int)
        for year in range(min_year, max_year):
            time_constraint.end_dt = time_constraint.end_dt.replace(year=year)

            for user_id in user_list:
                rmse = Accuracy.rmse(self.trainset.predict_movies_watched(user_id=user_id, n=n_movies, k=k,
                                                                          time_constraint=time_constraint))
                if abs(rmse - no_constraint_data[user_id]) < max_diff:
                    time_constraint_data[year] += 1

        return time_constraint_data

    def evaluate_max_year_constraint(self, n_users, n_movies, k, time_constraint):
        """
        Compare given time_constraint with normal where no constraint exists.

        Time constraint is of type max_year which means the system will be set to a certain year.

        :param n_users: Number of users to evaluate
        :param n_movies: Number of movies per user to evaluate
        :param k: Number of neighbours to take into account when making movie prediction
        :param time_constraint: Time constraint which will be applied.
        :return: DataFrame of results which contains rmse with constraint and no constraint, as well as runtime.
        """
        trainset = self.trainset
        data = list()

        for i in range(n_users):
            # Get Random User
            user_id = random.randint(1, 610)
            # Predict movies for user and record runtime
            st = default_timer()
            rmse = Accuracy.rmse(
                trainset.predict_movies_watched(user_id=user_id, n=n_movies, k=k, time_constraint=None))
            r1 = default_timer() - st
            # Predict movies with time_constraint for user and record runtime
            st = default_timer()
            time_constrained_rmse = Accuracy.rmse(
                trainset.predict_movies_watched(user_id=user_id, n=n_movies, k=k, time_constraint=time_constraint))
            r2 = default_timer() - st
            # Save iteration data
            data.append([user_id, rmse, r1, time_constrained_rmse, r2])

        data = pd.DataFrame(data)
        data.columns = ['user_id', 'rmse', 'runtime1', 'temporal_rmse', 'runtime2']
        data.set_index('user_id', inplace=True)
        return data

    def evaluate_time_bins_in_bulk(self, n, n_users, k=10,
                                   min_year=-1,
                                   max_year=-1,
                                   min_time_bin_size=2, max_time_bin_size=10):
        """
        Evaluate time bins and return the results.

        This method calls 'evaluate_time_bins' method 'n' times. Takes required precautions for bulk calling.

        :param n: Number of runs
        :param n_users: Number of users
        :param k: Number of neighbours will be used when making prediction
        :param min_year: First year to start when taking time bins
        :param max_year: When to stop when taking time bins, last is not included.
        :param min_time_bin_size: Minimum bin size in years
        :param max_time_bin_size: Maximum bin size in years
        :return: Evaluation results
        """
        if min_year == -1:
            min_year = self.trainset.trainset_user.get_first_timestamp().year

        if max_year == -1:
            max_year = datetime.now().year

        # Cache all years before processing
        time_constraint = TimeConstraint(start_dt=datetime(year=min_year, month=1, day=1),
                                         end_dt=datetime(year=max_year, month=1, day=1))
        self.trainset.similarity.cache_user_corrs_in_bulk_for_time_bins(time_constraint,
                                                                        min_year=min_year,
                                                                        max_year=max_year,
                                                                        min_time_bin_size=min_time_bin_size,
                                                                        max_time_bin_size=max_time_bin_size)

        run_results = dict()
        for i in range(n):
            run_results[i] = self.evaluate_time_bins(n_users=n_users, k=k, min_year=min_year, max_year=max_year,
                                                     min_time_bin_size=min_time_bin_size,
                                                     max_time_bin_size=max_time_bin_size,
                                                     create_cache=False)

        return run_results

    def evaluate_time_bins(self, n_users, k, min_year=-1, max_year=-1,
                           min_time_bin_size=2, max_time_bin_size=10,
                           create_cache=True) -> dict:
        """

        :param n_users: Number of users
        :param k: Number of neighbours will be used when making prediction
        :param min_year: First year to start when taking time bins
        :param max_year: When to stop when taking time bins, last is not included.
        :param min_time_bin_size: Minimum bin size in years
        :param max_time_bin_size: Maximum bin size in years
        :param create_cache: Create cache before calling time bins. For bulk callers.
        :return:
        """
        trainset = self.trainset

        if min_year == -1:
            min_year = self.trainset.trainset_user.get_first_timestamp().year

        if max_year == -1:
            max_year = datetime.now().year

        if n_users > 600:
            user_list = trainset.trainset_user.get_users()
        else:
            user_list = trainset.trainset_user.get_random_users(n=n_users)
        user_movie_list = trainset.trainset_movie.get_random_movie_per_user(user_list)
        data = dict()

        result = list()

        if create_cache:
            # Cache all years before processing
            time_constraint = TimeConstraint(start_dt=datetime(year=min_year, month=1, day=1),
                                             end_dt=datetime(year=max_year, month=1, day=1))
            self.trainset.similarity.cache_user_corrs_in_bulk_for_time_bins(time_constraint,
                                                                            min_year=min_year,
                                                                            max_year=max_year,
                                                                            min_time_bin_size=min_time_bin_size,
                                                                            max_time_bin_size=max_time_bin_size)

        # Take each bins where first bin 'min_time_bin_size' years, last one 'max_time_bin_size - 1' years
        for time_bin_size in range(min_time_bin_size, max_time_bin_size):
            # Shift each time_bin starting with 0 years up until (time_bin-1) years
            for shift in range(0, time_bin_size):
                curr_year = min_year + shift
                predictions = list()
                start_time = default_timer()
                # Scan and make predictions for all the time_bins
                while (curr_year + time_bin_size) < max_year:
                    for user_id, movie_id in user_movie_list:
                        p = trainset.predict_movie(user_id=user_id, movie_id=movie_id, k=k,
                                                   time_constraint=TimeConstraint(start_dt=datetime(curr_year, 1, 1),
                                                                                  end_dt=datetime(curr_year+time_bin_size, 1, 1)),
                                                   bin_size=time_bin_size)
                        # if prediction has been done successfully
                        if p != 0:
                            r = trainset.trainset_movie.get_movie_rating(movie_id=movie_id, user_id=user_id)
                            # Append (prediction, actual_rating)
                            predictions.append((p, r))
                    curr_year += time_bin_size
                runtime = default_timer() - start_time
                bin_rmse = Accuracy.rmse(predictions=predictions)
                iteration_results = {"bin_size": time_bin_size,
                                     "start_year": min_year + shift,
                                     "predictions": predictions,
                                     "rmse": bin_rmse,
                                     "runtime": runtime
                                     }
                result.append(iteration_results)

        data['result'] = result
        return data

In [9]:
c = TemporalCache(time_constraint=None, 
                  is_ratings_cached=True,
                  is_movies_cached=True,
                  is_movie_ratings_cached=True,
                  ratings=MovieLensDataset.load_ratings(r'C:\Users\Yukawa\datasets\ml-latest-small\ratings.csv'),
                  movies=MovieLensDataset.load_movies(r'C:\Users\Yukawa\datasets\ml-latest-small\movies.csv'),
                  movie_ratings=MovieLensDataset.load(),
                  is_user_correlations_cached=True,
                  use_bulk_corr_cache=True)

In [10]:
t = Trainset(cache=c, min_common_elements=5)
e = Evaluator(trainset=t)

## Timebin Based Predictions

In [48]:
class TimebinSimilarity:
    
    def __init__(self, ratings, trainset, k=5, r=3, s=10, p=3, corr_threshold=0.5, 
                 neighbour_min_s=5, neighbour_max_s=50, neighbour_step_size=5):
        """
        :param k: Minimum number of ratings in common in between neighbour users when taking timebins of them. 
        :param r: Minimum number of ratings in common between neighbour timebins.
        :param s: Maximum number of movies to include inside of the timebin.
        :param p: Minimum number of neighbour timebins when making prediction.
        :param neighbour_min_s: Minimum number of movies to include inside neighbour timebins
        :param neighbour_max_s: Maximum number of movies to include inside neighbour timebins
        :param neighbour_step_size: Number of movies to extend per step the timebin size
        """
        
        self.ratings = ratings
        self.ratings.sort_values(['user_id', 'timestamp'], ignore_index=True, inplace=True)
        self.trainset = trainset
        self.k = k
        self.r = r
        self.s = s
        self.p = p
        self.corr_threshold = corr_threshold
        
        self.neighbour_min_s = neighbour_min_s
        self.neighbour_max_s = neighbour_max_s
        self.neighbour_step_size = neighbour_step_size
        
        if s < 1:
            raise Exception("Timebin size must be positive.")
        
        # For ease of use, save these
        self.trainset_user = trainset.trainset_user
        self.trainset_movie = trainset.trainset_movie
        self.first_timestamp = ratings['timestamp'].min()
                
        # Set these before calculating any correlations
        self.user_id = None
        self.timebin = None
        self.timebin_user_avg_rating = None
       
    def find_timebin_corr(self, merged:pd.DataFrame, neighbour_avg_rating):
        """
        Find the correlation between the given neighbour timebin and the self.timebin
        
        :param merged: Merged version of neighbour timebin and self.timebin
        """
        
        # Calculate Pearson Correlation in between the self.timebin and given neighbour
        numenator = ((merged['rating_x'] - self.timebin_user_avg_rating) * (merged['rating_y'] - neighbour_avg_rating)).sum()
        denominator = math.sqrt(((merged['rating_x'] - self.timebin_user_avg_rating) ** 2).sum())
        denominator *= math.sqrt(((merged['rating_y'] - neighbour_avg_rating) ** 2).sum())
        pearson = numenator / denominator
        
        return pearson
        
    
    def get_movies_watched(self, user_id: int, time_constraint: TimeConstraint = None) -> pd.DataFrame:
        """
        Get all the movies watched by the chosen user.

        :param user_id: the user that we want to get the movies he-she has watched.
        :param time_constraint: type of the time constraint.
        :return: DataFrame of all movies watched with 'item_id', 'rating' columns
        """

        movie_ratings = self.ratings

        if time_constraint is None:
            return movie_ratings.loc[(movie_ratings['user_id'] == user_id)][['item_id', 'rating', 'timestamp']].set_index('item_id')

        if time_constraint.is_valid_max_limit():
            return movie_ratings.loc[(movie_ratings['user_id'] == user_id)
                                     & (movie_ratings.timestamp < time_constraint.end_dt)][['item_id', 'rating', 'timestamp']].set_index('item_id')
        elif time_constraint.is_valid_time_bin():
            return movie_ratings.loc[(movie_ratings['user_id'] == user_id)
                                     & (movie_ratings.timestamp >= time_constraint.start_dt)
                                     & (movie_ratings.timestamp < time_constraint.end_dt)][['item_id', 'rating', 'timestamp']].set_index('item_id')
        return None   # Means the Time_constraint is not a valid timebin!
    
    def get_timebin2(self, user_id, timebin_i, timebin_size):
        all_movies_watched = self.get_movies_watched(user_id)
        return all_movies_watched.iloc[timebin_i:timebin_i+timebin_size]
    
    def get_timebin_neighbours(self, user_id:int, timebin_i, timebin_size):
        """
        Get neighbours who has rated at least p movies in common with the user.
        """
        ratings = self.ratings
        
        # Create the identified user timebin
        timebin = self.get_timebin2(user_id, timebin_i, timebin_size)
        time_constraint = TimeConstraint(end_dt=timebin.iloc[len(timebin)-1]['timestamp'])
        
        # Count number of common ratings with other users
        userlist = [0 for i in range(611)]
        for movie_id in timebin.index.values.tolist():
            users_who_watched = ratings.loc[(ratings['item_id'] == movie_id) & (ratings['timestamp'] < time_constraint.end_dt)][['user_id']].values.tolist()
            for user_who_watched in users_who_watched:
                userlist[user_who_watched[0]] += 1

        # save as neighbour, if common rating count greater than k
        neighbour_id_list = []
        for i in range(0, 611):
            if userlist[i] > self.k:
                neighbour_id_list.append(i)
        return neighbour_id_list
    
    
    def get_neighbour_timebins(self, neighbour_id:int)->list:
        """
        Given a neighbour, get its list of timebins
        """
        # Start by taking all movies watched by the neighbour
        all_movies_watched = self.get_movies_watched(neighbour_id)
        n_movies = len(all_movies_watched)
        neighbour_timebins = list()
        
        # For each timebin_size
        for timebin_size in range(self.neighbour_min_s, self.neighbour_max_s, self.neighbour_step_size):
            # Traverse the all movies by taking 'timebin_size' piece of ratings per loop
            for i in range(0, n_movies, timebin_size):
                # Insert the timebin, its index i, and its size timebin_size into the list as a tuple
                neighbour_timebins.append(  (all_movies_watched.iloc[i:i+timebin_size], i, timebin_size)  )
        
        return neighbour_timebins
    
    def get_most_similar_timebins(self, user_id, timebin_i, timebin_size):
        """
        Get Neighbour Timebins
        """
        
        # Create the user timebin
        timebin = self.get_timebin2(user_id, timebin_i, timebin_size)
        time_constraint = TimeConstraint(end_dt=timebin.iloc[len(timebin)-1]['timestamp'])
        # Set the self.timebin as the timebin of interest with required details for correlation calculations
        self.timebin = timebin
        self.user_id = user_id
        self.timebin_user_avg_rating = self.trainset_user.get_user_avg_at(user_id, time_constraint.end_dt)
        
        # Neighbours contains users who has rated self.k movies in common
        neighbours = self.get_timebin_neighbours(user_id, timebin_i, timebin_size)
        data = list()
        
        # For Each neighbour
        
        for neighbour_id in neighbours:
            neighbour_timebins = self.get_neighbour_timebins(neighbour_id) 
            neighbour_avg_rating = self.trainset_user.get_user_avg_at(neighbour_id, time_constraint.end_dt)
            
            # For each neighbour timebin, calculate pearson between the timebin and neighbour timebin 
            for neighbour_timebin, timebin_i, timebin_size in neighbour_timebins:
                #print(neighbour_timebin)
                # Filter unrelated movie ratings and only keep common movie ratings
                
                merged = neighbour_timebin.merge(self.timebin, on='item_id')
                common_elements = len(merged)
                
                # If less than self.r movies in common rated in the neighbour_timebin, dont process this timebin
                if common_elements < self.r:
                    continue
                
                # Calculate Pearson Correlation between neighbour timebin and self.timebin
                corr = self.find_timebin_corr(merged, neighbour_avg_rating)
                
                
                # Dont include the neighbour timebin if corr is less than corr_threshold
                if corr < self.corr_threshold:                     
                    continue
                
                binsize_in_days = TimebinSimilarity.get_timebin_size(
                                    TimeConstraint(start_dt=neighbour_timebin.iloc[0]['timestamp'], 
                                                   end_dt=neighbour_timebin.iloc[len(neighbour_timebin)-1]['timestamp']))
                
                if not math.isnan(corr):
                    data.append( (neighbour_id, common_elements, corr, timebin_i, timebin_size, binsize_in_days) )
        
        return pd.DataFrame(data, columns=['neighbour_id', 'n_common','pearson_corr', 'timebin_i', 'timebin_size', 'timebin_size_in_days'])
    
    
    def get_timebin_neighbours_data(self, timebin, similar_timebins):
        """
        Get timebin possessors' ratings and timebin correlations as the output
        """
        # Store data in order to return as result
        data = defaultdict(list)
        for row in similar_timebins.itertuples(index=False):
            # Get neighbour timebin data
            neighbour_id = row[0]
            n_common = row[1]
            corr = row[2]
            timebin_i = row[3]
            timebin_size = row[4]
            timebin_size_in_days = row[5]
            
            # Create the neighbour timebin from its data
            neighbour_bin = self.get_timebin2(neighbour_id, timebin_i, timebin_size)
            
            # Get rid of movie ratings of the neighbour that are not found in the timebin
            merged_bin = pd.merge(timebin, neighbour_bin, left_index=True, right_index=True)
            
            # For each common movie rating, store neighbour rating and its correlation to the timebin
            for bin_row in merged_bin.itertuples(index=True):
                curr_movie = bin_row[0]
                neighbour_rating = bin_row[3]
                #print(bin_row[0], bin_row[1], bin_row[2], bin_row[3], bin_row[4])
                data[curr_movie].append( (neighbour_rating, corr, neighbour_id, n_common, timebin_i, timebin_size, timebin_size_in_days) )
        return data
    
    
    def compare_normal_and_timebin_predictions(self, n_users):
        """
        Compare n number of users
        
        """
        
        # list of ((normal_rmse, timebin_rmse),user_id, timebin_i, timebin_size, data) tuples
        output = list()
        
        # collected data count
        count = 0
        
        while count < n_users:

            # Choose a random user
            user_id = random.randint(0,610)
            
            ### Choose Random timebin
            
            # Get all movies that has been watched by the user
            movies_watched = self.get_movies_watched(user_id)
            
            # take a random starting point and take self.s piece movies starting from that point
            max_movie_index = len(movies_watched)-self.s
            if max_movie_index <= 0:
                continue
            timebin_i = random.randint(0, max_movie_index)
            timebin = self.get_timebin2(user_id, timebin_i, self.s)
            timebin_size = len(timebin)
            
            if timebin is None or len(timebin) < self.r:
                continue

            ### Make prediction without timebins
            
            # Make all the movies inside of the timebin by the classic way,using pearson correlation
            normal_predictions = TimebinSimilarity.predict_movies_watched(timebin=timebin, trainset = self.trainset, 
                                                                                 user_id=user_id)
            normal_rmse = Accuracy.rmse(normal_predictions)
            normal_threshold_accuracy  = Accuracy.threshold_accuracy(normal_predictions)
            
            # In case we are not able to make predictions without using timebins
            # dont make any further processing since we can not compare 0 with timebin rmse 
            if normal_rmse == 0:
                continue
            
            ### Make prediction with timebins

            # Get similart timebins
            
            similar_timebins = self.get_most_similar_timebins(user_id, timebin_i, timebin_size)
            
            # Drop duplicate (neighbour_id, n_common, pearson_corr, timebin_i) tuples so that now no repeating timebin exists
            similar_timebins.drop_duplicates(['neighbour_id','n_common', 'pearson_corr', 'timebin_i'], inplace=True)
            # Sort the timebins for the purpose of debugging
            #similar_timebins.sort_values('pearson_corr', ascending=False, inplace=True, ignore_index=True)
            
            data = self.get_timebin_neighbours_data(timebin, similar_timebins)
                
            predictions = TimebinSimilarity.predict_movies_watched_using_timebin_neighbours(data, 
                                                                                            self.trainset_movie, 
                                                                                            user_id, 
                                                                                            min_neighbour_count=self.p)

            timebins_rmse = Accuracy.rmse(predictions)
            timebin_threshold_accuracy = Accuracy.threshold_accuracy(predictions)
            
            # In case timebin_rmse is 0 continue since we can not compare with normal rmse
            if timebins_rmse == 0:
                continue
            
            # Debug output
            print(f"Normal RMSE: {normal_rmse:.2} Timebin RMSE:{timebins_rmse:.2} \
                    Normal Threshold Acc: {int(normal_threshold_accuracy)}% Timebin Threshold Acc: {int(timebin_threshold_accuracy)}%")

            # Save the data
            output.append( ((normal_rmse, timebins_rmse), 
                            (normal_threshold_accuracy, timebin_threshold_accuracy), 
                            user_id, timebin_i, timebin_size, 
                            data) )

            # we saved a data succesfully, increment collected data cout 
            count += 1
            
        return output
    
    @staticmethod
    def predict_movies_watched_using_timebin_neighbours(data, trainset_movie, user_id, min_neighbour_count=5):
        predictions = list()
        
        for movie_id, rating_corr_list in data.items():
            weighted_sum = 0
            weight_sum = 0
            count = 0
            for rating_corr in rating_corr_list:
                count += 1
                rating = rating_corr[0]
                corr = rating_corr[1]
                weighted_sum += rating * corr
                weight_sum += corr
            if count < min_neighbour_count:         # if less than min_neighbour_count neighbour found, pass
                continue
            prediction = weighted_sum / weight_sum
            actual = trainset_movie.get_movie_rating(movie_id=movie_id, user_id=user_id)
            predictions.append( (prediction, actual) )
        return predictions
    
    @staticmethod
    def predict_movies_watched_using_timebin_neighbours2(data, trainset_movie, user_id, min_neighbour_count=5):
        predictions = list()
        
        for movie_id, rating_corr_list in data.items():
            weighted_sum = 0
            weight_sum = 0
            count = 0
            for rating_corr in rating_corr_list:
                count += 1
                rating = rating_corr[0]
                corr = rating_corr[1]
                weighted_sum += rating * corr
                weight_sum += corr
            if count < min_neighbour_count:         # if less than min_neighbour_count neighbour found, pass
                continue
            prediction = weighted_sum / weight_sum
            actual = trainset_movie.get_movie_rating(movie_id=movie_id, user_id=user_id)
            predictions.append( (movie_id, (prediction, actual) ) )
        return predictions
    
    @staticmethod
    def predict_movies_watched(timebin, trainset, user_id):
        movies_watched = timebin.index.to_list()
        predictions = list()
        for movie in movies_watched:
            prediction = trainset.predict_movie(user_id=user_id, movie_id=movie, k=10)
            
            if prediction != 0:
                actual = trainset.trainset_movie.get_movie_rating(movie_id=movie, user_id=user_id)
                predictions.append((prediction, actual))
        return predictions
    
    @staticmethod
    def get_timebin_size(tc: TimeConstraint):
        return abs((tc.start_dt - tc.end_dt).days)

In [49]:
dataset = MovieLensDataset(is_movies_cached=True, is_ratings_cached=True)

In [50]:
timebin_similarity = TimebinSimilarity(ratings=dataset.ratings, trainset=t, s=10)

In [51]:
prediction_data = timebin_similarity.compare_normal_and_timebin_predictions(10)

Normal RMSE: 1.1 Timebin RMSE:0.083                     Normal Threshold Acc: 88% Timebin Threshold Acc: 88%
Normal RMSE: 0.54 Timebin RMSE:0.1                     Normal Threshold Acc: 100% Timebin Threshold Acc: 100%
Normal RMSE: 0.1 Timebin RMSE:0.075                     Normal Threshold Acc: 100% Timebin Threshold Acc: 100%
Normal RMSE: 1.4 Timebin RMSE:0.15                     Normal Threshold Acc: 66% Timebin Threshold Acc: 90%
Normal RMSE: 0.7 Timebin RMSE:0.025                     Normal Threshold Acc: 100% Timebin Threshold Acc: 90%
Normal RMSE: 0.45 Timebin RMSE:0.23                     Normal Threshold Acc: 80% Timebin Threshold Acc: 90%
Normal RMSE: 0.25 Timebin RMSE:0.12                     Normal Threshold Acc: 100% Timebin Threshold Acc: 100%
Normal RMSE: 0.29 Timebin RMSE:0.15                     Normal Threshold Acc: 66% Timebin Threshold Acc: 80%
Normal RMSE: 0.14 Timebin RMSE:0.25                     Normal Threshold Acc: 100% Timebin Threshold Acc: 100%
Normal RMSE:

In [61]:
import pickle

In [62]:
results = list()
for i in range(1,9):
    with open(f"v4_compare_normal_and_timebin_predictions_{i}.data", "rb") as handle:
        result = pickle.loads(handle.read())
        results.append(result)

In [63]:
def bulk_timebin_results_to_runs(bulk_results):
    runs = list()
    for bulk in bulk_results:
        for run in bulk:
            runs.append(run)
    return runs

In [64]:
run_results = bulk_timebin_results_to_runs(results)  # 1300x

In [65]:
len(run_results)

1300

In [66]:
predictions = TimebinSimilarity.predict_movies_watched_using_timebin_neighbours(run_results[1299][5], 
                                                                                timebin_similarity.trainset_movie, 
                                                                                149, 
                                                                                min_neighbour_count=3)
print("predicted - actual")
predictions

predicted - actual


[(1.0, 1.0),
 (4.0, 4.0),
 (3.772225870116762, 4.0),
 (1.3717847562660883, 1.0),
 (1.7435695125321764, 1.0),
 (3.0, 3.0),
 (2.2720781033494273, 2.0),
 (4.0, 4.0)]

In [81]:
Accuracy.analize(predictions)

{'accuracy': [1.0, 0.667, 0.75, 0.875, 0.875, 1.0, 1.0, 1.0, 1.0, 1.0],
 'balanced_accuracy': [0.5, 0.667, 0.375, 0.5, 0.438, 1.0, 0.5, 1.0, 0.5, 0.5],
 'informedness': [0.0, 0.333, -0.25, 0.0, -0.125, 1.0, 0.0, 1.0, 0.0, 0.0],
 'markedness': [0.0, 0.6, 0.0, -0.125, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0],
 'f1': [0, 0.5, 0, 0, 0, 1.0, 0, 1.0, 0, 0],
 'mcc': [0, 0.447, 0, 0, 0, 1.0, 0, -1.0, 0, 0],
 'precision': [0, 1.0, 0.0, 0, 0.0, 1.0, 0, 1.0, 0, 0],
 'recall': [0, 0.333, 0, 0.0, 0, 1.0, 0, 1.0, 0, 0],
 'specificity': [1.0, 1.0, 0.75, 1.0, 0.875, 1.0, 1.0, 1.0, 1.0, 1.0],
 'NPV': [1.0, 0.6, 1.0, 0.875, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}

In [83]:
Accuracy.threshold_analize(predictions)

{'accuracy': 1.0,
 'balanced_accuracy': 1.0,
 'informedness': 1.0,
 'markedness': 1.0,
 'f1': 1.0,
 'mcc': 1.0,
 'precision': 1.0,
 'recall': 1.0,
 'specificity': 1.0,
 'NPV': 1.0}

In [ ]:
predictions = TimebinSimilarity.predict_movies_watched_using_timebin_neighbours(run_results[1299][5], 
                                                                                timebin_similarity.trainset_movie, 
                                                                                run_results[1299][2], 
                                                                                min_neighbour_count=3)
print("predicted - actual")
predictions

In [89]:
run_results[1299]

((0.6, 0.09375),
 (60.0, 87.5),
 149,
 33,
 10,
 defaultdict(list,
             {1253: [(5.0, 0.9999999999999998, 149, 10, 30, 15, 1),
               (5.0, 1.0, 149, 7, 30, 10, 1)],
              1270: [(2.0, 0.9999999999999998, 149, 10, 30, 15, 1),
               (2.0, 1.0, 149, 7, 30, 10, 1)],
              1274: [(1.0, 0.9999999999999998, 149, 10, 30, 15, 1),
               (1.0, 1.0000000000000002, 149, 8, 35, 35, 1),
               (1.0, 1.0, 149, 7, 30, 10, 1),
               (1.0, 1.0, 149, 5, 35, 5, 1)],
              1334: [(4.0, 0.9999999999999998, 149, 10, 30, 15, 1),
               (4.0, 1.0000000000000002, 149, 8, 35, 35, 1),
               (4.0, 1.0, 149, 7, 30, 10, 1),
               (4.0, 1.0, 149, 5, 35, 5, 1)],
              1356: [(4.0, 0.9999999999999998, 149, 10, 30, 15, 1),
               (4.0, 1.0000000000000002, 149, 8, 35, 35, 1),
               (4.0, 1.0, 149, 7, 30, 10, 1),
               (3.0, 0.5579193713344573, 314, 6, 105, 35, 184),
               (4.0, 1

In [96]:
big_analysis = list()
for i in range(1300):
    predictions = TimebinSimilarity.predict_movies_watched_using_timebin_neighbours(run_results[i][5], 
                                                                                timebin_similarity.trainset_movie, 
                                                                                run_results[i][2], 
                                                                                min_neighbour_count=3)
    big_analysis.append( Accuracy.analize(predictions) )

C:\Users\Yukawa\Anaconda3\lib\site-packages\ipykernel_launcher.py:172: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\Yukawa\Anaconda3\lib\site-packages\ipykernel_launcher.py:172: RuntimeWarning: invalid value encountered in double_scalars


In [91]:
for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [98]:
def find_list_avg(l):
    return sum(l) / len(l)

In [97]:
big_analysis

[{'accuracy': [1.0, 0.9, 0.9, 1.0, 1.0, 0.833, 0.75, 0.667, 0.9, 1.0],
  'balanced_accuracy': [0.5,
   0.5,
   0.45,
   0.5,
   1.0,
   0.833,
   0.857,
   0.75,
   0.45,
   0.5],
  'informedness': [0.0, 0.0, -0.1, 0.0, 1.0, 0.667, 0.714, 0.5, -0.1, 0.0],
  'markedness': [0.0, -0.1, 0.0, 0.0, 1.0, 0.75, 0.333, 0.5, 0.0, 0.0],
  'f1': [0, 0, 0, 0, 1.0, 0.8, 0.5, 0.667, 0, 0],
  'mcc': [0, 0, 0, 0, 1.0, 0.707, 0.488, 0.5, 0, 0],
  'precision': [0, 0, 0.0, 0, 1.0, 1.0, 0.333, 1.0, 0.0, 0],
  'recall': [0, 0.0, 0, 0, 1.0, 0.667, 1.0, 0.5, 0, 0],
  'specificity': [1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 0.714, 1.0, 0.9, 1.0],
  'NPV': [1.0, 0.9, 1.0, 1.0, 1.0, 0.75, 1.0, 0.5, 1.0, 1.0]},
 {'accuracy': [1.0, 1.0, 1.0, 1.0, 0.9, 0.875, 0.9, -inf, 0.9, 1.0],
  'balanced_accuracy': [0.5,
   0.5,
   0.5,
   0.5,
   0.45,
   0.75,
   0.45,
   0.857,
   0.45,
   1.0],
  'informedness': [0.0, 0.0, 0.0, 0.0, -0.1, 0.5, -0.1, 0.714, -0.1, 1.0],
  'markedness': [0.0, 0.0, 0.0, 0.0, 0.0, 0.857, 0.0, 1.4, 0.0, 1.

In [ ]:
for i in range(1300):
    for i in range(10):

In [99]:
big_analysis[0]

{'accuracy': [1.0, 0.9, 0.9, 1.0, 1.0, 0.833, 0.75, 0.667, 0.9, 1.0],
 'balanced_accuracy': [0.5,
  0.5,
  0.45,
  0.5,
  1.0,
  0.833,
  0.857,
  0.75,
  0.45,
  0.5],
 'informedness': [0.0, 0.0, -0.1, 0.0, 1.0, 0.667, 0.714, 0.5, -0.1, 0.0],
 'markedness': [0.0, -0.1, 0.0, 0.0, 1.0, 0.75, 0.333, 0.5, 0.0, 0.0],
 'f1': [0, 0, 0, 0, 1.0, 0.8, 0.5, 0.667, 0, 0],
 'mcc': [0, 0, 0, 0, 1.0, 0.707, 0.488, 0.5, 0, 0],
 'precision': [0, 0, 0.0, 0, 1.0, 1.0, 0.333, 1.0, 0.0, 0],
 'recall': [0, 0.0, 0, 0, 1.0, 0.667, 1.0, 0.5, 0, 0],
 'specificity': [1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 0.714, 1.0, 0.9, 1.0],
 'NPV': [1.0, 0.9, 1.0, 1.0, 1.0, 0.75, 1.0, 0.5, 1.0, 1.0]}

In [102]:
for k, v in big_analysis[0].items():
    if k == 'accuracy':
        print(k,v)

accuracy [1.0, 0.9, 0.9, 1.0, 1.0, 0.833, 0.75, 0.667, 0.9, 1.0]


In [114]:
big_analysis_avg = defaultdict(float)

for i in range(1300):
    for k, v in big_analysis[i].items():
        avg = find_list_avg(v)
        if math.isnan(avg) or math.isinf(avg):
            print("hey")
        big_analysis_avg[k] += avg  if not math.isnan(avg) and not math.isinf(avg) else 0

for k, v in big_analysis_avg.items():
    big_analysis_avg[k] /= 1300
    
big_analysis_avg

hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey
hey


defaultdict(float,
            {'accuracy': 0.8159031538461543,
             'balanced_accuracy': 0.5364036153846141,
             'informedness': 0.07279846153846141,
             'markedness': 0.08357830769230783,
             'f1': 0.15760392307692275,
             'mcc': 0.05133430769230756,
             'precision': 0.1773300769230766,
             'recall': 0.1615191538461534,
             'specificity': 0.9112809999999998,
             'NPV': 0.906249769230768})

In [117]:
prediction_data[9]

((0.34375, 0.075),
 (100.0, 100.0),
 116,
 41,
 10,
 defaultdict(list,
             {1291: [(5.0, 0.6236793397982505, 68, 3, 100, 20, 1),
               (5.0, 0.6236793397982505, 68, 3, 90, 30, 1),
               (5.0, 0.6396869137633832, 68, 4, 90, 45, 1),
               (4.0, 0.8668167384091158, 96, 3, 10, 10, 1),
               (4.0, 0.8668167384091158, 96, 3, 0, 20, 1),
               (4.0, 1.0, 116, 5, 45, 5, 1),
               (4.0, 0.9999999999999999, 116, 9, 40, 10, 1),
               (4.0, 1.0, 116, 6, 45, 15, 1),
               (4.0, 1.0, 116, 10, 40, 20, 1),
               (4.0, 0.9999999999999999, 116, 9, 25, 25, 1),
               (4.0, 1.0, 116, 10, 30, 30, 1),
               (4.0, 1.0, 116, 10, 35, 35, 1),
               (4.0, 0.9040737377931777, 144, 3, 90, 10, 1),
               (4.0, 0.5743131074540568, 144, 4, 80, 20, 1),
               (4.0, 0.7140714833398674, 144, 6, 75, 25, 1),
               (4.0, 0.7140714833398674, 144, 6, 70, 35, 1),
               (3.5, 0.85

In [119]:
analysis = list()
for i in range(10):
    predictions = TimebinSimilarity.predict_movies_watched_using_timebin_neighbours(prediction_data[i][5], 
                                                                                timebin_similarity.trainset_movie, 
                                                                                prediction_data[i][2], 
                                                                                min_neighbour_count=3)
    analysis.append( Accuracy.analize(predictions) )
analysis

C:\Users\Yukawa\Anaconda3\lib\site-packages\ipykernel_launcher.py:172: RuntimeWarning: divide by zero encountered in double_scalars


[{'accuracy': [1.0, 1.0, 1.0, 0.889, 0.889, 0.714, 0.714, 1.0, 1.0, 1.0],
  'balanced_accuracy': [0.5,
   0.5,
   0.5,
   0.5,
   0.444,
   0.667,
   0.833,
   1.0,
   0.5,
   1.0],
  'informedness': [0.0, 0.0, 0.0, 0.0, -0.111, 0.333, 0.667, 1.0, 0.0, 1.0],
  'markedness': [0.0, 0.0, 0.0, -0.111, 0.0, 0.667, 0.333, 1.0, 0.0, 1.0],
  'f1': [0, 0, 0, 0, 0, 0.5, 0.5, 1.0, 0, 1.0],
  'mcc': [0, 0, 0, 0, 0, 0.471, 0.471, 1.0, 0, 1.0],
  'precision': [0, 0, 0, 0, 0.0, 1.0, 0.333, 1.0, 0, 1.0],
  'recall': [0, 0, 0, 0.0, 0, 0.333, 1.0, 1.0, 0, 1.0],
  'specificity': [1.0, 1.0, 1.0, 1.0, 0.889, 1.0, 0.667, 1.0, 1.0, 1.0],
  'NPV': [1.0, 1.0, 1.0, 0.889, 1.0, 0.667, 1.0, 1.0, 1.0, 1.0]},
 {'accuracy': [1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 0.75, 1.0, 0.7, 0.7],
  'balanced_accuracy': [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 0.35, 0.5],
  'informedness': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -0.3, 0.0],
  'markedness': [0.0, 0.0, 0.0, 0.0, 0.0, -0.1, -0.25, 1.0, 0.0, -0.3],
  'f1': [0, 0, 0, 0, 0, 0

In [121]:
analysis_avg = defaultdict(float)

for i in range(10):
    for k, v in analysis[i].items():
        avg = find_list_avg(v)
        if math.isnan(avg) or math.isinf(avg):
            print("hey")
        analysis_avg[k] += avg  if not math.isnan(avg) and not math.isinf(avg) else 0

for k, v in analysis_avg.items():
    analysis_avg[k] /= 10
    
analysis_avg

hey


defaultdict(float,
            {'accuracy': 0.83206,
             'balanced_accuracy': 0.59057,
             'informedness': 0.18117,
             'markedness': 0.20657999999999999,
             'f1': 0.23544,
             'mcc': 0.12923,
             'precision': 0.25226999999999994,
             'recall': 0.24958,
             'specificity': 0.9315900000000001,
             'NPV': 0.9543099999999999})